# extract text and its position

In [1]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
import pdfminer
class pdfPositionHandling:

    def parse_obj(self, lt_objs, pageNum, contents):

        # loop over the object list
        for obj in lt_objs:

            if isinstance(obj, pdfminer.layout.LTTextLine):
                #print("%6d, %6d, %s" % (obj.bbox[0], obj.bbox[1], obj.get_text().replace('\n', '_')))
                text = obj.get_text().replace('\n','')
                if text in contents:
                    contents[text].append((obj.bbox[0], obj.bbox[1], pageNum))
                else:
                    contents[text] = [(obj.bbox[0], obj.bbox[1], pageNum)]

            # if it's a textbox, also recurse
            if isinstance(obj, pdfminer.layout.LTTextBoxHorizontal):
                self.parse_obj(obj._objs, pageNum, contents)

            # if it's a container, recurse
            elif isinstance(obj, pdfminer.layout.LTFigure):
                self.parse_obj(obj._objs, pageNum, contents)

    def parsepdf(self, filename, startpage, endpage):

        # Open a PDF file.
        fp = open(filename, 'rb')

        # Create a PDF parser object associated with the file object.
        parser = PDFParser(fp)

        # Create a PDF document object that stores the document structure.
        # Password for initialization as 2nd parameter
        document = PDFDocument(parser)

        # Check if the document allows text extraction. If not, abort.
        if not document.is_extractable:
            raise PDFTextExtractionNotAllowed

        # Create a PDF resource manager object that stores shared resources.
        rsrcmgr = PDFResourceManager()

        # Create a PDF device object.
        device = PDFDevice(rsrcmgr)

        # BEGIN LAYOUT ANALYSIS
        # Set parameters for analysis.
        laparams = LAParams()

        # Create a PDF page aggregator object.
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)

            # Create a PDF interpreter object.
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        contents = {}
        i = 0
        # loop over all pages in the document
        for page in PDFPage.create_pages(document):
            if i >= startpage and i <= endpage:
                #print('------------')
                #print('page' + str(i))
                # read the page into a layout object
                interpreter.process_page(page)
                layout = device.get_result()

                # extract text from this object
                self.parse_obj(layout._objs, i, contents)
            i += 1
        return contents

# set paths

In [2]:
#from Tkinter import *
#import tkFileDialog
from tkinter import *
import tkinter.filedialog

win = Tk()
win.title('Select the PDF file')
var = StringVar()

w = Label(win, text="File Path:")
e = Entry(win, textvariable=var)
#b = Button(win, text="Browse",command=lambda:var.set(tkFileDialog.askopenfilename()))
b = Button(win, text="Browse",command=lambda:var.set(tkinter.filedialog.askopenfilename()))
w.pack(side=LEFT)
e.pack(side=LEFT)
b.pack(side=LEFT)
win.mainloop()

path1 = var.get()

win = Tk()
win.title('Select the output file\'s direnctory')
var = StringVar()

w = Label(win, text="File Path:")
e = Entry(win, textvariable=var)
#b = Button(win, text="Browse",command=lambda:var.set(tkFileDialog.askdirectory()))
b = Button(win, text="Browse",command=lambda:var.set(tkinter.filedialog.askdirectory()))
w.pack(side=LEFT)
e.pack(side=LEFT)
b.pack(side=LEFT)
win.mainloop()

path2 = var.get()+'/'

In [3]:
#path2 = 'C:/Users/zhongzhu/Documents/GitHub/work/temp/'
#path1 = 'C:/Users/zhongzhu/Documents/GitHub/work/Seattle.pdf'
print(path1, path2)

C:/Users/zhongzhu/Documents/GitHub/work/CORE.pdf C:/Users/zhongzhu/Documents/GitHub/work/temp/


# record positions

In [4]:
record_pos = pdfPositionHandling()


In [5]:
contents = record_pos.parsepdf(path1, 0, 50)
#for pos in contents:
    #print(pos)
    #print(contents[pos])

In [6]:
#import Tkinter
#import tkSimpleDialog
import tkinter
import tkinter.simpledialog

#root = Tkinter.Tk()
root = tkinter.Tk()
root.title('select sample pages')
var1 = tkinter.StringVar()
var2 = tkinter.StringVar()
#b1 = Button(root, text="Select start page",command=lambda:var1.set(tkSimpleDialog.askinteger("Select start page", "enter page number", parent = root)))
#b2 = Button(root, text="Select end page",command=lambda:var2.set(tkSimpleDialog.askinteger("Select end page", "enter page number", parent = root)))
b1 = tkinter.Button(root, text="Select start page",command=lambda:var1.set(tkinter.simpledialog.askinteger("Select start page", "enter page number", parent = root)))
b2 = tkinter.Button(root, text="Select end page",command=lambda:var2.set(tkinter.simpledialog.askinteger("Select end page", "enter page number", parent = root)))
b1.grid(column = 0, row = 0)
b2.grid(column = 1, row = 0)
#var = tkSimpleDialog.askstring("Select Pates", "enter your name", parent = root)
root.mainloop()
start_page = var1.get()
end_page = var2.get()

In [7]:
# get the positions of some sample pages
first_page_contents = record_pos.parsepdf(path1, startpage=int(start_page), endpage=int(end_page))

In [8]:
#for i in first_page_contents:
    #print(i)
    #print(first_page_contents[i])

In [9]:
filted_contents = {}
for text in contents:
    if len(contents[text]) > 20:  ###set at beginning#####################
        filted_contents[text] = contents[text]
        #print(text + ' ' +str(len(filted_contents[text])))

In [10]:
#print(filted_contents)

In [11]:
#from Tkinter import *
#import Tkinter as tk
import tkinter as tk

class Example(tk.Frame):
     
    def __init__(self, root, *args, **kwargs):
        tk.Frame.__init__(self, root, *args, **kwargs)
        self.root = root
        self.vsb = tk.Scrollbar(self, orient="vertical")
        self.text = tk.Text(self, width=40, height=20, yscrollcommand=self.vsb.set)
        self.vsb.config(command=self.text.yview)
        self.vsb.pack(side="right", fill="y")
        self.text.pack(side="left", fill="both", expand=True)
        
        self.temp_contents = {}

        var = []
        c = []
        for i, cat in enumerate(sorted(filted_contents)):
            c.append(cat)
            v = tk.BooleanVar()
            var.append(v)
            cb = tk.Checkbutton(self, text="#%s" % cat, variable=var[i])
            self.text.window_create("end", window=cb)
            self.text.insert("end", "\n") # to force one checkbox per line
                      
        def var_states():
            for i,v in enumerate(var):
                if v.get():
                    #print(c[i])
                    self.temp_contents[c[i]] = filted_contents[c[i]]
        tk.Button(root, text='Select', command=var_states).pack()
        
root = tk.Tk()
root.title('selecte categories')
win = Example(root)
win.pack(side="top", fill="both", expand=True)
root.mainloop()

filted_contents = win.temp_contents

# search neighbors

In [22]:
from itertools import product
neighbors = {}
    
for line1, line2 in product(filted_contents, first_page_contents):
    key = line1
    value = line2
    if not key in neighbors:
        neighbors[key] = []
    if(line1 != line2 and line1 in first_page_contents
       and abs(int(first_page_contents[line1][0][0])-int(first_page_contents[line2][0][0]))<500    ###set at beginning
       and abs(int(first_page_contents[line1][0][1])-int(first_page_contents[line2][0][1]))<100): ###set at beginning
        neighbors[key].append(value)

In [23]:
#for key in neighbors:
    #print(key, neighbors[key])

# GUI

In [24]:
#import Tkinter as tk
#import ttk
import tkinter as tk
import tkinter.ttk

win = tk.Tk()
win.title("Set Pairs")   

#ttk.Label(win, text="Select category:").grid(column=0, row=0)   
tkinter.ttk.Label(win, text="Select category:").grid(column=0, row=0) 
#ttk.Label(win, text="content starts from").grid(column=2, row=0) 
#ttk.Label(win, text="content ends before").grid(column=3, row=0)     
pos_or_area = StringVar()
def select():  
    if pos_or_area.get() == 'area':
        ls = list(neighbors.keys())   
        ls += ['#END']
        dropdown2['values'] = ls
    else:
        ls = neighbors[category.get()]
        dropdown2['values'] = ls
    dropdown2.current()    

R1 = tkinter.ttk.Radiobutton(win, text="find by position", variable=pos_or_area, value='pos', command=select)
R1.grid(column = 2, row = 0)
R2 = tkinter.ttk.Radiobutton(win, text="find by area", variable=pos_or_area, value='area', command=select)
R2.grid(column = 3, row = 0)
    
pairs = {}
def bind():   
    #key = contents[category.get()]
    #new_value = contents[content.get()]
    key = category.get()
    if not key in pairs:
        pairs[key] = set([])
    
    if pos_or_area.get()=='pos':
        a = first_page_contents[category.get()][0]
        b = first_page_contents[content.get()][0]
        c = (b[0]-a[0], b[1]-a[1], b[2], pos_or_area.get())
        pairs[key].add(c)
    else:
        c = (content.get(), pos_or_area.get()) 
        pairs[key].add(c)
        
    dropdown2.current()
    
# creat a drop-down list
category = tk.StringVar()
dropdown1 = tkinter.ttk.Combobox(win, width=40, height=20, textvariable=category)
dropdown1['values'] = list(neighbors.keys())   
dropdown1.grid(column=0, row=1)      
#dropdown1.current(0)     


# create another drop-down list
content = tk.StringVar()
dropdown2 = tkinter.ttk.Combobox(win, width=40, textvariable=content)    
dropdown2.grid(column=2, row=1)  
 

# bind button
action = tkinter.ttk.Button(win, text="Bind", command=bind)
action.grid(column=4, row=1) 

win.mainloop()      # window

In [25]:
#for i in pairs:
    #print(i)
    #print(type(pairs[i]))
    #print((pairs[i]))
    #for i in pairs[i]:
        #print(len(i))

In [26]:
print(pairs)

{'Critical Path Work Effected?': {('#END', 'area')}, 'Date:': {(46.7999999999999, 0.0, 3, 'pos')}, 'Day:': {(51.44999999999993, 0.0, 3, 'pos')}, 'Fair Working Days To Date:': {(168.79999999999995, -23.25, 3, 'pos')}, 'High': {(5.199999999999932, -11.600000000000023, 3, 'pos')}, 'Is Today Claimed as Weather Day?': {(169.54999999999995, 2.25, 3, 'pos')}, 'Job:': {(48.299999999999955, 0.0, 3, 'pos')}, 'Location:': {(46.54999999999994, 1.3000000000000682, 3, 'pos')}, 'Low': {(4.2999999999999545, -11.600000000000023, 3, 'pos')}, 'Notes: ': {('#END', 'area')}, 'Precipitation Amount: ': {('Precipitation Amount: ', 'area')}, 'Project:': {(46.54999999999994, 0.0, 3, 'pos')}, 'Remarks: ': {('Notes: ', 'area')}, 'Report #:': {(88.09999999999991, 0.0, 3, 'pos')}, 'Temp:': {('Temp:', 'area')}, 'Weather Condition:': {(102.80000000000001, 2.25, 3, 'pos')}}


In [27]:
content_map = contents = record_pos.parsepdf(path1, 0, 50)

In [18]:
print(content_map['WEATHER'])

KeyError: 'WEATHER'

In [ ]:
print(content_map['LABOR HOURS'])

In [28]:
def find_content_by_pos(content_map, content_pos):
    content = {}
    for c in content_map:
        m = 1 #### maximal error allowed#########################
        for pos in content_map[c]:
            error = abs(content_pos[0]-pos[0]) + abs(content_pos[1]-pos[1])
            #print error
            if error < m and content_pos[-1]==pos[-1]:
                if pos[1] in content:
                    content[pos[1]].append((c, pos[0])) 
                else:
                    content[pos[1]] = [(c, pos[0])]
    #print(m)
    return content

def find_content_by_area(content_map, end_cat, start_pos):
    content = {}
    end_pos = (0,0,start_pos[-1]);
    
    if end_cat in content_map:
        for pos in content_map[end_cat]:
            if pos[-1] == start_pos[-1]:
                end_pos = pos
                break
    for c in content_map:
        for pos in content_map[c]:
            #print(pos, start_pos,end_pos)
            if pos[-1]==start_pos[-1] and pos[1] > end_pos[1] and pos[1] < start_pos[1]:
                if pos[1] in content:
                    content[pos[1]].append((c, pos[0])) 
                else:
                    content[pos[1]] = [(c, pos[0])]
    return content
    
data = {}

for cat in pairs:
    print(pairs[cat])
    #if cat in content_map:  #unnecessary??
    pages = {}
    for pos in content_map[cat]:
        content = {}
        for shift in pairs[cat]:
            if shift[-1] == 'pos':
                content_pos = (pos[0]+shift[0], pos[1]+shift[1], pos[-1])
                content.update(find_content_by_pos(content_map, content_pos))
            elif shift[-1] == 'area':
                end_cat = shift[0]
                #if end_cat == '#END':
                    #continue #more?
                content = find_content_by_area(content_map, end_cat, pos)
        pages[pos[-1]] = content
    data[cat] = pages

{('#END', 'area')}
{(46.7999999999999, 0.0, 3, 'pos')}
{(51.44999999999993, 0.0, 3, 'pos')}
{(168.79999999999995, -23.25, 3, 'pos')}
{(5.199999999999932, -11.600000000000023, 3, 'pos')}
{(169.54999999999995, 2.25, 3, 'pos')}
{(48.299999999999955, 0.0, 3, 'pos')}
{(46.54999999999994, 1.3000000000000682, 3, 'pos')}
{(4.2999999999999545, -11.600000000000023, 3, 'pos')}
{('#END', 'area')}
{('Precipitation Amount: ', 'area')}
{(46.54999999999994, 0.0, 3, 'pos')}
{('Notes: ', 'area')}
{(88.09999999999991, 0.0, 3, 'pos')}
{('Temp:', 'area')}
{(102.80000000000001, 2.25, 3, 'pos')}


In [29]:
def hierarchical(paragraph):
    if len(paragraph) == 0:
        return {}
    else:
        leftest = 10000
        for line_pos, row in paragraph.items():
            for col in row:
                leftest = min(leftest, col[1])
    new_para = {}
    #select subcategories
    for line_pos, row in paragraph.items():
        for col in row:
            if col[1]-leftest<1:              #######
                new_para[(col[0], line_pos, col[1])] = {}
                break;
    #restructure
    for line_pos, row in paragraph.items():
        selected_cat = ()
        min_dist = 10000
        for cat in new_para:
            if line_pos <= cat[1] and cat[1] - line_pos < min_dist:
                min_dist = cat[1] - line_pos
                selected_cat = cat
        if len(selected_cat) > 0:
            new_para[selected_cat][line_pos] = row;
        else:
            new_para[(row[0][0], line_pos, row[0][1])] = {}
            #new_para[(row[0][0], line_pos)][line_pos] = row
    
    return new_para

In [30]:
new_data = {}
for cat, values in data.items():
    new_data[cat] = {}
    for page_num, paragraph in values.items():
        new_data[cat][page_num] = hierarchical(paragraph)

In [31]:
print(new_data)

{'Critical Path Work Effected?': {0: {('1', 18.20260000000002, 60.5): {18.20260000000002: [('1', 60.5)]}, ('Sub', 558.744, 153.75): {548.244: [('Wrkrs', 150.95), ('Hrs', 178.49999999999997)], 546.744: [('Wrkrs', 256.3), ('Hrs Work Accomplished', 286.5)], 547.85495: [('2nd Tier', 201.0)], 557.244: [('Tier', 259.15000000000003), ('Tier', 286.0)], 549.1432: [('S', 546.0), ('Q Sch', 564.8)]}, ('2nd', 567.744, 259.05): {}}, 1: {('Remarks: ', 308.2932, 20.25): {308.2932: [('Remarks: ', 20.25)]}, ('Notes: ', 296.2932, 20.25): {296.2932: [('Notes: ', 20.25)], 18.20260000000002: [('2', 60.5)]}}, 2: {('Communication with Subcontractors: Met with Malone Electric.', 548.9932, 19.999999999999996): {548.9932: [('Communication with Subcontractors: Met with Malone Electric.', 19.999999999999996)]}, ('Met with Discount Permits, set up SWPPP.', 537.0432, 19.999999999999986): {537.0432: [('Met with Discount Permits, set up SWPPP.', 19.999999999999986)]}, ('Visitors: Met with Entergy Onsite', 524.59319999

In [ ]:
print(data['LABOR HOURS'])

# generate txt table

In [105]:
from operator import itemgetter
f = open(path2+'table_CORE_area.txt', 'w', newline = '')

for cat, contents in data.items():
    f.write(cat + '\n')
    #print(contents)
    print(cat)
    if type(next (iter (contents.values()))) is dict:
        #print(contents)
        for pageNum, page in contents.items():
            f.write('\n' + str(pageNum) + '\n')
            for linepos, line in reversed(sorted(page.items())):
                line.sort(key=itemgetter(1))
                for c in line:
                    f.write(c[0]+'\t|')
                f.write('\n')
            f.write('\n')
    else:
        for pageNum in contents:
            f.write('\n' + str(pageNum) + '\n')
            for line in contents[pageNum]:
                f.write(line + '\n')
f.close()

Critical Path Work Effected?
Date:
Day:
Job:
Location:
Project:
Report #:
Temp:


In [106]:
from operator import itemgetter
f = open(path2+'CORE_table_by_pages.txt', 'w', newline = '')
for pageNum in range(0,50):
    f.write('=====PAGE_'+str(pageNum+1) + '=====\n\n')
    for cat, contents in data.items():
        if pageNum in contents:
            f.write(cat + ':\n')
            #f.write('------\n')
            if type(next (iter (contents.values()))) is dict and pageNum in contents:
                for linepos, line in reversed(sorted(contents[pageNum].items())):
                    line.sort(key=itemgetter(1))
                    for c in line:
                        f.write(c[0]+'\t|\t')
                    f.write('\n')
                f.write('\n')
            elif pageNum in contents:
                for line in contents[pageNum]:
                    f.write(line + '\n')
                f.write('\n')
    f.write('======PAGE_'+str(pageNum+1) + ' END======\n\n')
f.close() 

# draw

In [24]:
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
from operator import itemgetter

font = ImageFont.truetype("obj4304.dump", 10)
img = Image.open("2.png")
draw = ImageDraw.Draw(img)
for pageNum in range(0,1):
    for cat, contents in data.items():
        if pageNum in contents:
            print(cat, contents[pageNum])
            if type(next (iter (contents.values()))) is dict:
                for linepos, line in reversed(sorted(contents[pageNum].items())):
                    line.sort(key=itemgetter(1))
                    for c in line:
                        #print(c[0])
                        draw.text((c[1],900-linepos) , c[0],(100,100,100), font = font) 
            elif pageNum in contents:
                for line in contents[pageNum]:
                    pass
                
img.save('new-out1.jpg')
img.close()



ACTIVITY START/FINISH {338.62535199999996: [('No activities were started or finished today', 85.6832634)]}
CONTRACT NUMBER {672.340829: [('DACA67-02-D-2012  0004', 424.14)]}
CONTRACTOR {649.0636895: [('Evergreene Construction, L.C.', 82.14336499999999)]}
DATE {694.9015785: [('01 Nov 2004 - Monday', 424.14)]}
EQUIPMENT HOURS {132.04409049999998: [('No equipment hours were Reported today', 85.6832634)], 118.8442295: [('Total operating hours to date: 0.0', 95.5795366)]}
LABOR HOURS {203.203308: [('The following labor hours were Reported today:', 85.6832634)], 190.065491: [('Labor Classification', 197.10009846000003), ('Employer', 95.5872923), ('Employees', 441.0308237199999), ('Worked', 508.69853064999967)], 176.68368949999999: [('OPERATING ENGINEER', 197.07031462999998), ('0.0', 464.3325122), ('0.0', 532.0087504), ('Asbestos Abatement', 95.58)], 163.30068749999998: [('0.0', 197.08069851999994), ('Total hours worked to date:', 95.5849548999999)], 166.12368949999998: [('0.0', 464.337467099

In [25]:
font = ImageFont.truetype("obj4304.dump", 10)
img = Image.open("2.png")
draw = ImageDraw.Draw(img)

for cat in pairs:
    for pos in content_map[cat]:
        if pos[-1] == pageNum:
            draw.text((pos[0],900-pos[1]) , cat,(0,0,0), font = font) 
            
img.save('new-out2.jpg')
img.close()

In [26]:
font = ImageFont.truetype("obj4304.dump", 10)
img = Image.open("2.png")
draw = ImageDraw.Draw(img)

for pageNum in range(0,1):
    for cat, contents in data.items():
        if pageNum in contents:
            print(cat, contents[pageNum])
            if type(next (iter (contents.values()))) is dict:
                for linepos, line in reversed(sorted(contents[pageNum].items())):
                    line.sort(key=itemgetter(1))
                    for c in line:
                        #print(c[0])
                        draw.text((c[1],900-linepos) , c[0],(100,100,100), font = font) 
            elif pageNum in contents:
                pass
                #for line in contents[pageNum]:
                    #pass

for cat in pairs:
    for pos in content_map[cat]:
        if pos[-1] == pageNum:
            draw.text((pos[0],900-pos[1]) , cat,(0,0,0), font = font) 
          
img.save('new-out3.jpg')
img.close()

ACTIVITY START/FINISH {338.62535199999996: [('No activities were started or finished today', 85.6832634)]}
CONTRACT NUMBER {672.340829: [('DACA67-02-D-2012  0004', 424.14)]}
CONTRACTOR {649.0636895: [('Evergreene Construction, L.C.', 82.14336499999999)]}
DATE {694.9015785: [('01 Nov 2004 - Monday', 424.14)]}
EQUIPMENT HOURS {132.04409049999998: [('No equipment hours were Reported today', 85.6832634)], 118.8442295: [('Total operating hours to date: 0.0', 95.5795366)]}
LABOR HOURS {203.203308: [('The following labor hours were Reported today:', 85.6832634)], 190.065491: [('Employer', 95.5872923), ('Labor Classification', 197.10009846000003), ('Employees', 441.0308237199999), ('Worked', 508.69853064999967)], 176.68368949999999: [('Asbestos Abatement', 95.58), ('OPERATING ENGINEER', 197.07031462999998), ('0.0', 464.3325122), ('0.0', 532.0087504)], 163.30068749999998: [('Total hours worked to date:', 95.5849548999999), ('0.0', 197.08069851999994)], 166.12368949999998: [('Total', 417.78), ('

In [33]:
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
from operator import itemgetter

font = ImageFont.truetype("obj4304.dump", 10)
img = Image.open("2.png")
draw = ImageDraw.Draw(img)

for pageNum in range(3,4):
    for cat, contents in new_data.items():
        if pageNum in contents:
            #if type(next (iter (contents.values()))) is dict:
            for subcat, lines in contents[pageNum].items():
                #line.sort(key=itemgetter(1))

                for line_pos, line in lines.items():
                    for c in line:
                        draw.text((c[1],900-line_pos) , c[0],(200,200,200), font = font) 
                draw.text((subcat[2], 900-subcat[1]), subcat[0], (100,100,100), font = font)
                print(subcat)
                
for cat in pairs:
    for pos in content_map[cat]:
        if pos[-1] == pageNum:
            draw.text((pos[0],900-pos[1]) , cat,(0,0,0), font = font) 
          
img.save('core-out3.jpg')
img.close()

('Bayou Tree Service Inc.', 481.094, 20.25)
('Remarks: ', 389.5932, 20.25)
('Notes: ', 377.5932, 20.25)
('Employee', 630.40495, 20.25)
('Tanner Broughton', 615.144, 20.25)
('Tanner Broughton', 572.244, 20.25)
('Dading, Marques & Associates LLC', 467.794, 20.25)
('Phoenix Constructors LLC', 454.49399999999997, 20.25)
('Phoenix Constructors LLC', 441.19399999999996, 20.25)
('Ace Fence & Gate LLC', 427.89399999999995, 20.25)
('George Caillouet', 600.844, 20.25)
('George Caillouet', 557.944, 20.25)
('Chuck Hutchinson', 586.544, 20.25)
('Chuck Hutchinson', 529.3439999999999, 20.25)
('Chris Cole', 543.644, 20.25)
('Subcontractor', 495.80494999999996, 20.25)
('10/09/2014', 705.6432, 557.15)
('Thursday', 691.9431999999999, 564.4)
('N', 680.8932, 189.8)
(' 90', 666.0432, 582.5)
('N', 680.8932, 189.8)
('14-02-002', 734.1432, 560.7)
('4801 Maid Marion Drive', 722.1432, 67.55)
(' 78', 666.0432, 553.0)
('4', 18.20260000000002, 60.5)
('Sherwood Forest Elementary School', 734.1432, 67.55)
(' 1', 720.